In [ ]:
import gym
import numpy as np
import random
import math
from time import sleep

## CartPole Environment

### Details
* Name: CartPole-v0  
* Category: Classic Control
* [Leaderboard Page](https://github.com/openai/gym/wiki/Leaderboard#cartpole-v0)
* Old links:
  * [Environment Page](https://gym.openai.com/envs/CartPole-v0)  
  * [Algorithms Page](https://gym.openai.com/algorithms?groups=classic_control)

### Description
A pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The pendulum starts upright, and the goal is to prevent it from falling over by increasing and reducing the cart's velocity.

### Source
This environment corresponds to the version of the cart-pole problem described by Barto, Sutton, and Anderson 


In [ ]:
env = gym.make('CartPole-v0')

### Observation
Type: Box(4)

Num | Observation | Min | Max
---|---|---|---
0 | Cart Position | -2.4 | 2.4
1 | Cart Velocity | -Inf | Inf
2 | Pole Angle | ~ -41.8&deg; | ~ 41.8&deg;
3 | Pole Velocity At Tip | -Inf | Inf

### Starting State
All observations are assigned a uniform random value between ±0.05

### Code

> #### Box Space는 Continuous Space
> - Q-Network를 구성한다면, state가 Continuous한 상태로 문제를 풀 수 있으나, Q-table을 이용할 경우 discrete하게 나눌 필요
> - 이 경우, reward 산정에 필요한 state는 **Pole Angle**과 **Pole Velocity At Tip**
> - Cart관련 state는 구간을 나눌 필요 없음
> - **Pole Angle**은 6개의 구간으로 나눔
> - **Pole Velocity At Tip**은 3개의 구간으로 나눔
> - 나눈 구간은 Q-table작성시 이용

In [ ]:
NUM_BUCKETS = (1, 1, 6, 3)
STATE_BOUNDS = list(zip(env.observation_space.low, env.observation_space.high))
STATE_BOUNDS[1] = [-0.5, 0.5]
STATE_BOUNDS[3] = [-math.radians(50), math.radians(50)]

### Actions
Type: Discrete(2)

Num | Action
--- | ---
0 | Push cart to the left
1 | Push cart to the right

In [ ]:
NUM_ACTIONS = env.action_space.n

### Reward
Reward is 1 for every step taken, including the termination step

### Code

> #### Q-table은 state 경우의 수 * discrete action 경우의 수 만큼 필요
> - (1, 1, 6, 3, 2)차원의 array를 Q-table로 활용
> - Q-table 0으로 초기화

In [ ]:
q_table = np.zeros(NUM_BUCKETS + (NUM_ACTIONS,))

### Episode Termination
1. Pole Angle is more than ±12°
2. Cart Position is more than ±2.4 (center of the cart reaches the edge of the display)
3. Episode length is greater than 200

### Solved Requirements
Considered solved when the average reward is greater than or equal to 195.0 over 100 consecutive trials.

### Code

> #### 학습 변수 세팅
> - 최소 epsilon = 0.01
> - 최소 학습률 = 0.1
> - 진행 episode = 1000 episode
> - episode length = 250 timestep
> - 성공조건 = timestep 200 달성
> - 종료조건 = 120회 연속으로 성공

In [ ]:
MIN_EXPLORE_RATE = 0.01
MIN_LEARNING_RATE = 0.1

NUM_EPISODES = 1000
MAX_T = 250
SOLVED_T = 199
STREAK_TO_END = 120

DEBUG_MODE = True

### Decay되는 변수(explore rate, learning rate) 함수 정의

- min, max clip
- $y(t) = 1 - \log{{t+1}\over{25}}$

<img src="./img/decay_graph.png" alt="graph" width="450" align="left"/>

In [ ]:
def get_explore_rate(t):
    if t >= 24:
        return max(MIN_EXPLORE_RATE, min(1, 1.0 - math.log10((t+1)/25)))
    else:
        return 1.0
    
def get_learning_rate(t):
    if t >= 24:
         return max(MIN_LEARNING_RATE, min(0.5, 1.0 - math.log10((t+1)/25)))
    else:
         return 1.0

### State Discretization

- state를 bucket index로 mapping
- bound 기준으로 벗어나는 값들은 양 끝단 index로
- bound내를 bucket수만큼의 구간으로 나누어 mapping

In [ ]:
def state_to_bucket(state):
    bucket_indice = []
    for i in range(len(state)):
        if state[i] <= STATE_BOUNDS[i][0]:
            bucket_index = 0
        elif state[i] >= STATE_BOUNDS[i][1]:
            bucket_index = NUM_BUCKETS[i] - 1
        else:
            bound_width = STATE_BOUNDS[i][1] - STATE_BOUNDS[i][0]
            offset = (NUM_BUCKETS[i]-1)*STATE_BOUNDS[i][0]/bound_width
            scaling = (NUM_BUCKETS[i]-1)/bound_width
            bucket_index = int(round(scaling*state[i] - offset))
        bucket_indice.append(bucket_index)
    return tuple(bucket_indice)

### Policy

- $\epsilon$-greedy
- $\epsilon=$ explore_rate

In [ ]:
def select_action(state, explore_rate):
    if random.random() < explore_rate:
        action = env.action_space.sample()
    else:
        action = np.argmax(q_table[state])
    return action

### Algorithm

- Q-learning (off-policy)
    - greedy action을 대상으로 업데이트함

    ```python
best_q = np.amax(q_table[state])  
q_table[state_0 + (action,)] += 
    learning_rate * (reward + discount_factor * (best_q) - q_table[state_0 + (action,)])  
```

- SARSA일 경우? (on-policy)
    - policy에 따라 선택한 action을 대상으로 업데이트함
    ```python
next_action = select_action(state, explore_rate) 
policy_q = q_table[state][next_action]
q_table[state_0 + (action,)] += 
    learning_rate * (reward + discount_factor * (policy_q) - q_table[state_0 + (action,)])  
```

In [ ]:
def simulate():
    
    learning_rate = get_learning_rate(0) 
    # learning rate 초기값 설정
    
    explore_rate = get_explore_rate(0) 
    # explore rate 초기값 설정
    
    discount_factor = 0.99  
    # discount factor 설정

    num_streaks = 0 
    # 연속성공 카운터

    for episode in range(NUM_EPISODES): 
    # NUM_EPISODES만큼 episode 진행

        obv = env.reset() 
        # state 초기화
        
        state_0 = state_to_bucket(obv) 
        # state discretization

        for t in range(MAX_T): 
        # episode length = MAX_T
        
            env.render() 
            # environment 렌더링 (그림)

            action = select_action(state_0, explore_rate) 
            # action 선택
            
            obv, reward, done, _ = env.step(action) 
            # 선택한 action environment에서 시행, state/reward 획득
            
            state = state_to_bucket(obv) 
            # state discretization

            best_q = np.amax(q_table[state]) 
            # 해당 state (전이된 state)에서 Q최대값 선택
            
            q_table[state_0 + (action,)] += learning_rate*(reward + discount_factor*(best_q) - q_table[state_0 + (action,)])
            # Q최대값을 이용하여 Q-table 업데이트 (이전 state)

            state_0 = state 
            # state 업데이트

            if (DEBUG_MODE):
                print("\nEpisode = %d" % episode)
                print("t = %d" % t)
                print("Action: %d" % action)
                print("State: %s" % str(state))
                print("Reward: %f" % reward)
                print("Best Q: %f" % best_q)
                print("Explore rate: %f" % explore_rate)
                print("Learning rate: %f" % learning_rate)
                print("Streaks: %d" % num_streaks)
                print("")
                # logging

            if done:
                print("%f 번의 steps 이후에 Episode %d이 끝났음" % (t, episode))
                if (t >= SOLVED_T):
                    num_streaks += 1
                    # 성공조건 만족시 연속성공 카운팅
                else:
                    num_streaks = 0
                    # 성공조건 불만족시 카운터 초기화
                break


        if num_streaks > STREAK_TO_END:
            break
            # 종료조건 만족시 종료

        explore_rate = get_explore_rate(episode)
        learning_rate = get_learning_rate(episode)
        # 진행 episode에 맞는 explore rate 및 learning rate로 업데이트


In [ ]:
simulate()